# Test McpExecutor Run Method

This notebook tests the `run` method of the `McpExecutor` class from `autocoder.common.mcp_tools`.

In [1]:
import os
import sys
import json
import asyncio
import tempfile
from pathlib import Path

from autocoder.common.mcp_hub import McpHub, McpServer, McpTool, McpResource, McpResourceTemplate
from autocoder.common.mcp_tools import McpExecutor
import byzerllm

In [2]:
# Helper function for async tests
async def run_test(test_func):
    """Helper to run async test functions"""
    try:
        await test_func()
        print("✅ Test passed")
    except AssertionError as e:
        print(f"❌ Test failed: {str(e)}")
    except Exception as e:
        print(f"❌ Test error: {str(e)}")

In [3]:
import json
async def test_mcp_executor_run():
    """Test McpExecutor run method"""
    # Create settings file with filesystem server config
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        settings = {
            "mcpServers": {
                "filesystem": {
                    "command": "npx",
                    "args": [
                        "-y",
                        "@modelcontextprotocol/server-filesystem",
                        "/Users/allwefantasy/projects/tests"                        
                    ]
                }
            }
        }
        json.dump(settings, f)
        settings_path = f.name
    
    try:
        # Initialize hub and executor
        hub = McpHub(settings_path)
        await hub.initialize()
        
        llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
        executor = McpExecutor(hub, llm)
        
        # Create test conversation
        conversations = [{
            "role": "user",
            "content": "List the contents of /Users/allwefantasy/projects/tests"
        }]
        
        # Run the executor
        result = await executor.run(conversations)        
        print(json.dumps(result, indent=2,ensure_ascii=False))
               
        
    finally:
        # Cleanup
        await hub.shutdown()
        os.unlink(settings_path)

In [4]:
# Run the test
print("Testing McpExecutor run method:")
await test_mcp_executor_run()

Testing McpExecutor run method:


2025-01-02 18:52:36.231 | ERROR    | autocoder.common.mcp_hub:_fetch_resources:247 - Failed to fetch resources for filesystem: code=-32601 message='Method not found' data=None
2025-01-02 18:52:36.233 | ERROR    | autocoder.common.mcp_hub:_fetch_resource_templates:284 - Failed to fetch resource templates for filesystem: code=-32601 message='Method not found' data=None
2025-01-02 18:52:36.234 | INFO     | autocoder.common.mcp_hub:update_server_connections:200 - Connected to new MCP server: filesystem
2025-01-02 18:52:36.272 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2025-01-02 18:52:36,300	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-01-02 18:52:36,318	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 



TOOL USE

You have access to a set of tools that are executed upon the user's approval. You can use one tool per message, and will receive the result of that tool use in the user's response. You use tools step-by-step to accomplish a given task, with each tool use informed by the result of the previous tool use.

# Tool Use Formatting

Tool use is formatted using XML-style tags. The tool name is enclosed in opening and closing tags, and each parameter is similarly enclosed within its own set of tags. Here's the structure:

<tool_name>
<parameter1_name>value1</parameter1_name>
<parameter2_name>value2</parameter2_name>
...
</tool_name>

For example:

<read_file>
<path>src/main.js</path>
</read_file>

Always adhere to this format for the tool use to ensure proper parsing and execution.

# Tools

## use_mcp_tool
Description: Request to use a tool provided by a connected MCP server. Each MCP server can provide multiple tools with different capabilities. Tools have defined input schemas tha